In [187]:
import pandas as pd
from bqplot import ColorScale, ColorAxis, DateScale, LinearScale, Axis, Lines, Figure
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from ipywidgets import Label, VBox, HBox, Layout, Accordion, Dropdown
from bqplot.market_map import MarketMap
import os
from datetime import datetime
from iexfinance import get_historical_data
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

## RETRIEVE AND CLEAN DATA

In [31]:
etf_data = pd.read_excel('../us_etfs.xlsx').drop_duplicates()
etf_data.columns = [i.lower() for i in etf_data.columns]
etf_list = list(etf_data['ticker'])
def get_iex_data(stock_list, start, end):
    return_list = []
    for i in stock_list:
        print(i)
        df = pd.DataFrame(get_historical_data(i, start, end, output_format='pandas')).interpolate()
        df['ticker'] = i
        return_list.append(df)
    return return_list

In [34]:
full = get_iex_data(etf_list, start=None, end=None)

SPY
IVV
VTI
VOO
QQQ
VEA
EFA
IEFA
VWO
AGG
IJH
IWM
IEMG
IJR
IWF
VTV
VUG
IWD
BND
LQD
XLF
VNQ
EEM
VIG
GLD
BSV
VO
VB
TIP
VEU
IVW
XLK
VGT
DIA
VYM
MDY
VCSH
IWB
XLV
VCIT
XLE
IWR
DVY
USMV
VGK
EWJ
SCHF
PFF
XLY
SDY
RSP
ITOT
IVE
SCHX
HYG
SHV
EMB
VBR
SHY
VV
SCHB
XLI
BIV
VT
MBB
BNDX
VXUS
IWO
IWS
FLOT
IXUS
MINT
MTUM
IGSB
SCZ
IAU
IWN
IBB
TLT
MUB
AMLP
JNK
FDN
VOE
IWP
EZU
XLP
IWV
VBK
IEF
EFAV
ACWI
VHT
SCHA
VFH
SCHD
IJK
GDX
SPLV
XLU
BKLN
IEI
IJT
VMBS
SCHG
IJS
VXF
QUAL
GOVT
IJJ
ITA
VOT
DBEF
HDV
EFV
VNQI
EWZ
GUNR
IGIB
SCHP
DXJ
VSS
SCHM
VTIP
IUSG
XBI
PRF
SCHZ
PGX
HEDJ
FXI
KRE
OEF
SCHE
EEMV
XLB
FNDX
CWB
SLV
SCHV
INDA
SCHH
IUSV
SPSB
NEAR
IDV
IYR
DGRO
FVD
IYW
VDE
EMLC
GDXJ
MGK
FNDF
VPL
VDC
TQQQ
GSLC
EWT
FLRN
VIS
PCY
AAXJ
EWY
NOBL
SPIB
VLUE
SPYG
SJNK
SHM
EFG
XOP
VTEB
FPE
BIL
FNDA
DON
KBE
SRLN
HEFA
MCHI
SPDW
ACWV
VCR
EWG
FEZ
VGSH
AMJ
IHI
FBT
TOTL
SHYG
ACWX
SPAB
RWX
EWC
SCHR
SCHO
FV
XLRE
VPU
VAW
TFI
HYD
VGIT
SSO
FTSM
SPTM
IGF
RPG
PDBC
IEUR
SPHD
EWH
QTEC
DBC
IXN
VCLT
RWR
ICF
XT
FLGE
SLYG
DGRW
JPST
VOOG
IEV
IYH
FTE

In [35]:
def lstm_clean_data(data):
    for i in range(len(data)):
        data[i] = data[i].reset_index().dropna()
        data[i]['date'] = pd.to_datetime(data[i]['date'])
        data[i] = data[i].set_index('date')
#         data[i]['friday'] = ((((data[i]['date'].diff(-1).astype('int'))/86400000000000)+1)/(-2)).astype('int')
        data[i]['Reg_Target'] = data[i]['close'].shift(-1)
#         data[i] = pd.merge(data[i], prices, on='date')
    return data

def add_past(etf_list, times):
    for i in range(len(etf_list)):
        for n in times:
            etf_list[i]['{}day return'.format(n)] = -etf_list[i]['close'].diff(periods=n).round(3)
    return etf_list

In [36]:
clean_full = lstm_clean_data(full)
data = add_past(clean_full, [1, 5, 21, 252])

In [37]:
data[0]

,open,high,low,close,volume,ticker,Reg_Target,1day return,5day return,21day return,252day return
date,,,,,,,,,,,
2015-01-02,191.6852,192.1496,189.6419,190.8029,121465865,SPY,187.3570,NaN,NaN,NaN,NaN
2015-01-05,189.6326,189.8184,187.0134,187.3570,169632646,SPY,185.5923,3.446,NaN,NaN,NaN
2015-01-06,187.7007,188.2858,184.6960,185.5923,209151408,SPY,187.9050,1.765,NaN,NaN,NaN
2015-01-07,187.0784,188.2858,186.5768,187.9050,125346709,SPY,191.2394,-2.313,NaN,NaN,NaN
2015-01-08,189.4840,191.4809,189.4654,191.2394,147217784,SPY,189.7069,-3.334,NaN,NaN,NaN
2015-01-09,191.7038,191.7224,189.0196,189.7069,158567288,SPY,188.2208,1.532,1.096,NaN,NaN
2015-01-12,189.8555,190.0320,187.5428,188.2208,144396067,SPY,187.6914,1.486,-0.864,NaN,NaN
2015-01-13,189.5862,190.8493,186.2332,187.6914,214553306,SPY,186.5583,0.529,-2.099,NaN,NaN
2015-01-14,185.4344,186.7812,184.4313,186.5583,192991092,SPY,184.8492,1.133,1.347,NaN,NaN


In [38]:
sub_data = data[:10]

## RUN MODELS

In [39]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import timeit

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [145]:
def lstm_time_test_split(X, n_past, date):
    X = X.reset_index()
    scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    ticker = X['ticker'].iloc[0]
    x_train = X[X['date'] < date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])
    scaler.fit(x_train)
    x_test = X[X['date'] >= date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])[:-1]
    x_train = scaler.transform(x_train)
    x_train = np.reshape(x_train,(x_train.shape[0], n_past, x_train.shape[1]))
    x_test = scaler.transform(x_test)
    x_test = np.reshape(x_test,(x_test.shape[0], n_past, x_test.shape[1]))
    y_train = np.array(X[X['date'] < date]['Reg_Target'].drop(columns='date')).ravel().astype('float').reshape(-1,1)
    y_scaler.fit(y_train)
    y_train = y_scaler.transform(y_train)
    y_test = np.array(X[X['date'] >= date]['Reg_Target'].drop(columns='date')).ravel().astype('float')[:-1].reshape(-1,1)
    y_test = y_scaler.transform(y_test)
    x_holdout = X[X['date'] >= date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])[-1:]
    x_holdout = scaler.transform(x_holdout)
    x_holdout = np.reshape(x_holdout,(x_holdout.shape[0], n_past, x_holdout.shape[1]))
#     y_test = scaler.transform(y_test)
    return ticker, x_train, x_test, x_holdout, y_train, y_test, scaler, y_scaler

def build_step_model(x_train, y_train, epoc):
    model = Sequential()
    model.add(LSTM(50, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('relu'))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(x_train, y_train, epochs=epoc, batch_size=64, validation_split=.07, verbose=2,shuffle=False)
    return model, history

def yield_preds(model, scaler, x_test, x_holdout, y_test):
    yhat = model.predict(x_test)
    preds = scaler.inverse_transform(yhat)
    true = scaler.inverse_transform(y_test)
    today = model.predict(x_holdout)
    today_pred = scaler.inverse_transform(today)
    return preds, today_pred

def run_all_lstms(data, split, epoc):
    out = pd.DataFrame()
    tomorrow = pd.DataFrame()
    start = timeit.default_timer()
    for i in range(len(data)):
        ticker, x_train, x_test, x_holdout, y_train, y_test, scaler, y_scaler = lstm_time_test_split(data[i], 1, split)
        print('Model #: {}'.format(i))
        model, history = build_step_model(x_train, y_train, epoc)
        preds, future = yield_preds(model, y_scaler, x_test, x_holdout, y_test)
        out[ticker] = preds.flatten()
        tomorrow[ticker] = future.flatten()
    out = out.set_index(data[0][-len(out):].index)
    stop = timeit.default_timer()
    print('Time: ', stop - start)
    return out, tomorrow

## MAKE DASHBOARD INPUTS

In [49]:
def make_yesterday_mapframe(ref):
    yesterday_inputs = pd.DataFrame([ref[i].reset_index().iloc[-1] for i in range(len(ref))])
    mapframe = pd.merge(etf_data, yesterday_inputs, on='ticker').drop(columns = 'Reg_Target')
    return mapframe

In [51]:
mapframe_y = make_yesterday_mapframe(data)
mapframe_y.columns = ['ticker','Fund Name','issuer','Expense Ratio','Total Assets','spread (%)','Segment','date','Open','High','Low','Previous Close',
                    'volume','1 Day return', '1 Week return', '1 Month return','1 Year return']

In [41]:
def make_daily_mapframe(ref, value):
    mapframe = pd.DataFrame()
    for i in range(len(ref)):
        ticker = ref[i]['ticker'].iloc[0]
        mapframe[ticker] = ref[i][value]
#     mapframe = mapframe.set_index(ref[0]['date'])
    return mapframe

In [42]:
mapframe_daily_close = make_daily_mapframe(data, 'close')

In [178]:
mapframe_preds, future_preds = run_all_lstms(sub, '09-2018', 12)

Model #: 0
Train on 859 samples, validate on 65 samples
Epoch 1/12
 - 18s - loss: 0.1463 - val_loss: 0.5424
Epoch 2/12
 - 0s - loss: 0.0698 - val_loss: 0.2750
Epoch 3/12
 - 0s - loss: 0.0181 - val_loss: 0.0720
Epoch 4/12
 - 0s - loss: 0.0151 - val_loss: 0.0164
Epoch 5/12
 - 0s - loss: 0.0176 - val_loss: 0.0210
Epoch 6/12
 - 0s - loss: 0.0127 - val_loss: 0.0218
Epoch 7/12
 - 0s - loss: 0.0090 - val_loss: 0.0109
Epoch 8/12
 - 0s - loss: 0.0067 - val_loss: 0.0052
Epoch 9/12
 - 0s - loss: 0.0051 - val_loss: 0.0030
Epoch 10/12
 - 0s - loss: 0.0037 - val_loss: 0.0013
Epoch 11/12
 - 0s - loss: 0.0029 - val_loss: 7.0114e-04
Epoch 12/12
 - 0s - loss: 0.0025 - val_loss: 3.4963e-04
Model #: 1
Train on 859 samples, validate on 65 samples
Epoch 1/12
 - 15s - loss: 0.1518 - val_loss: 0.5880
Epoch 2/12
 - 0s - loss: 0.0803 - val_loss: 0.3393
Epoch 3/12
 - 0s - loss: 0.0274 - val_loss: 0.1221
Epoch 4/12
 - 0s - loss: 0.0202 - val_loss: 0.0402
Epoch 5/12
 - 0s - loss: 0.0260 - val_loss: 0.0400
Epoch 6/

## DASHBOARD MARKETMAP

In [190]:
col = ColorScale(scheme='Portland')
segments = mapframe_y['Segment'].values
market_map1 = MarketMap(names=etf_list,      
                       # basic data which needs to set for each map
                       ref_data=mapframe_y, groups=segments, 
                       cols=50, row_groups=8,
                       # Data frame which can be used for different properties of the map
                       # Axis and scale for color data
                       tooltip_fields=['Fund Name', 'Segment', 'Expense Ratio', 'Total Assets', 'Previous Close', '1 Day return', '1 Week return', '1 Month return', '1 Year return'],
                       scales={'color': col},
                       layout={'min_width': '1400px', 'min_height': '1000px'})
# market_map1.color = ['1day_change']
market_map1.font_style = {'font-size': '8px', 'fill':'black'}
market_map1.title = 'ETF Market Map'

selector = Dropdown(options=['Segment', '1 Day return', '1 Week return', '1 Month return', '1 Year return'], value='Segment', description='Color By:', disabled=False,)

sc_x = DateScale()
sc_y = LinearScale()
ax_x = Axis(scale=sc_x, grid_lines='dashed', label='Date')
ax_y = Axis(scale=sc_y, orientation='vertical', grid_lines='dashed', label='Price (USD)')
line = Lines(x= mapframe_daily_close['06-2018':].index.values, y=[], scales={'x': sc_x, 'y': sc_y}, colors=['black'])
linep = Lines(x= mapframe_preds.index.values, y=[], scales={'x': sc_x, 'y': sc_y}, colors=['orange'])

ts_output = Figure(marks=[line, linep], axes=[ax_x, ax_y], layout=Layout(width='88%', height='600px'))
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        market_map1.color = mapframe_y[change['new']]
        market_map1.axes = [ColorAxis(scale=col, label=change['new'], visible=True)]
        
def on_box_change(change):
    line.y = mapframe_daily_close[change['new'][-1]]['06-2018':]
    linep.y = mapframe_preds[change['new'][-1]]
    ts_output.title = '{} Price 2018'.format(change['new'][-1])
    
selector.observe(on_change)
market_map1.observe(on_box_change, 'selected')
first_page = VBox([selector, market_map1, ts_output])
first_page

VBox(children=(Dropdown(description='Color By:', options=('Segment', '1 Day return', '1 Week return', '1 Month return', '1 Year return'), value='Segment'), MarketMap(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], cols=50, font_style={'font-size': '8px', 'fill': 'black'}, groups=array(['Equity: U.S. - Large Cap', 'Equity: U.S. - Large Cap',
       'Equity: U.S. - Total Market', ...,
       'Equity: Germany - Total Market',
       'Equity: South Korea - Total Market',
       'Equity: U.K. - Total Market'], dtype=object), layout=Layout(min_height='1000px', min_width='1400px'), map_margin={'top': 50, 'right': 50, 'left': 50, 'bottom': 50}, names=array(['SPY', 'IVV', 'VTI', ..., 'DBGR', 'DBKO', 'DBUK'], dtype='<U4'), ref_data=     ticker                                          Fund Name  \
0       SPY                             SPDR S&P 500 ETF Trust   
1       IVV                           iShares Core S&P 500 ETF   
2       VTI                    Vanguard Total Stock Market ETF   
3       VOO                               Vanguard S&P 500 ETF   
4       QQQ                                  Invesco QQQ Trust   
5       VEA                Vanguard FTSE Developed Markets ETF   
6       EFA                              iShares MSCI EAFE ETF   
7      IEFA                         iShares Core MSCI EAFE ETF   
8       VWO                 Vanguard FTSE Emerging Markets ETF   
9       AGG               iShares Core U.S. Aggregate Bond ETF   
10      IJH                       iShares Core S&P Mid-Cap ETF   
11      IWM                           iShares Russell 2000 ETF   
12     IEMG             iShares Core MSCI Emerging Markets ETF   
13      IJR                     iShares Core S&P Small Cap ETF   
14      IWF                    iShares Russell 1000 Growth ETF   
15      VTV                                 Vanguard Value ETF   
16      VUG                                Vanguard Growth ETF   
17      IWD                     iShares Russell 1000 Value ETF   
18      BND                     Vanguard Total Bond Market ETF   
19      LQD  iShares iBoxx USD Investment Grade Corporate B...   
20      XLF                  Financial Select Sector SPDR Fund   
21      VNQ                           Vanguard Real Estate ETF   
22      EEM                  iShares MSCI Emerging Markets ETF   
23      VIG                 Vanguard Dividend Appreciation ETF   
24      GLD                                    SPDR Gold Trust   
25      BSV                       Vanguard Short-Term Bond ETF   
26       VO                               Vanguard Mid-Cap ETF   
27       VB                             Vanguard Small-Cap ETF   
28      TIP                              iShares TIPS Bond ETF   
29      VEU                  Vanguard FTSE All-World ex-US ETF   
...     ...                                                ...   
1616   SFIG  WisdomTree Fundamental U.S. Short-Term Corpora...   
1617   SFHY  WisdomTree Fundamental U.S. Short-Term High Yi...   
1618   HGSD    WisdomTree Global Hedged SmallCap Dividend Fund   
1619    GSD           WisdomTree Global SmallCap Dividend Fund   
1620   WCHN               WisdomTree ICBCCS S&P China 500 Fund   
1621   DWMF          WisdomTree International Multifactor Fund   
1622   DXJF            WisdomTree Japan Hedged Financials Fund   
1623   JHDG  WisdomTree Japan Hedged Quality Dividend Growt...   
1624   GULF               WisdomTree Middle East Dividend Fund   
1625   AGND  WisdomTree Negative Duration U.S. Aggregate Bo...   
1626    QSY     WisdomTree U.S. Quality Shareholder Yield Fund   
1627   SHAG  WisdomTree Yield Enhanced U.S. Short-Term Aggr...   
1628   EQLT                       Workplace Equality Portfolio   
1629   IFIX  Xtrackers Barclays International Corporate Bon...   
1630   IGVT  Xtrackers Barclays International Treasury Bond...   
1631   EMIH  Xtrackers Emerging Markets Bond Interest Rate ...   
1632   EURZ                  

In [167]:
Figure?

## REAL MARKETMAP

In [180]:
from fbprophet import Prophet as proph
def produce_prophet(ticker, timestep, conf):
    df = mapframeD[ticker]
    df = df.reset_index()
    df.columns = ['ds', 'y']
    p = proph(interval_width=.95, daily_seasonality=True, yearly_seasonality=True)
    p.fit(df)
    inp = p.make_future_dataframe(periods=timestep, freq='D')
    out = p.predict(inp)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    plot = p.plot(out, xlabel='Date', ylabel=ticker+' price(USD)')
    return out

In [394]:
mapframe_daily_close['DINT'].dropna().index

DatetimeIndex(['2018-03-02', '2018-03-05', '2018-03-06', '2018-03-07',
               '2018-03-08', '2018-03-09', '2018-03-12', '2018-03-13',
               '2018-03-14', '2018-03-15',
               ...
               '2018-09-18', '2018-09-19', '2018-09-20', '2018-09-21',
               '2018-09-24', '2018-09-25', '2018-09-26', '2018-09-27',
               '2018-09-28', '2018-10-01'],
              dtype='datetime64[ns]', name='date', length=148, freq=None)

In [445]:
# import plotly.plotly as py
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# import plotly.graph_objs as go
# from datetime import datetime

# def make_time_series(ticker):
#     df = mapframe_daily_close[ticker].dropna()
#     data = [go.Scatter(x=df.index, y=df.values)]
#     graph = plot(data)
#     return graph

# def make_candlestick(ticker):
#     df = mapframe_daily_close[ticker]
#     trace = go.Candlestick(x=df.Date, open=df.Open, high=df.High, low=df.Low, close=df.Close)
#     data = [trace]
#     py.iplot(data, filename='simple_candlestick')

In [467]:
col = ColorScale(scheme='Blured')
segments = mapframe['Segment'].values
ax_c = ColorAxis(scale=col, label='1-Day Change', visible=True)

market_map = MarketMap(names=hun_etfs, groups=segments,       # Basic data which needs to set for each map
                       cols=12, row_groups=12,
                       ref_data=mapframe,                               
                       tooltip_fields=['fund name', 'segment', 'open', 'high', 'low', 'close', '1 Day return', '7day return'],
                       tooltip_formats=['', '', '.1f'],
                       scales={'color': col}, axes=[ax_c],
                       layout=Layout(min_width='600px', min_height='600px'))
deb_output = Label()
sc_x = DateScale()
sc_y = LinearScale()
ax_x = Axis(scale=sc_x, grid_lines='dashed', label='Date')
ax_y = Axis(scale=sc_y, orientation='vertical', grid_lines='dashed', label='Price', label_location='end', label_offset='-1em')
line = Lines(x= mapframeD['2018':].index.values, y=[], scales={'x': sc_x, 'y': sc_y}, colors=['black'])
one_output = Figure(marks=[line], axes=[ax_x, ax_y])

def selected_index_changed(change):
    deb_output.value = str(change.new)
    line.y = mapframeD[change.new].values

market_map.color = mapframe['1 Day return']

market_map.observe(selected_index_changed, 'selected')
sidebar = VBox([deb_output, one_output])
heatmap = HBox([market_map, sidebar] )

accordion = Accordion(children=[first_page, heatmap])
accordion.set_title(0, 'Map Overview')
accordion.set_title(1, 'Heatmap')

accordion

/anaconda3/lib/python3.6/site-packages/bqplot/marks.py:235: DeprecationWarning:

Deprecated in traitlets 4.1, use the instance .metadata dictionary directly, like x.metadata[key] or x.metadata.get(key, default)



Accordion(children=(VBox(children=(Dropdown(description='Color By:', options=('Segment', '1 Day return', '1 Week return', '1 Month return', '1 Year return'), value='Segment'), MarketMap(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], cols=50, font_style={'font-size': '8px', 'fill': 'black'}, groups=array(['Equity: U.S. - Large Cap', 'Equity: U.S. - Large Cap',
       'Equity: U.S. - Total Market', ...,
       'Equity: Germany - Total Market',
       'Equity: South Korea - Total Market',
       'Equity: U.K. - Total Market'], dtype=object), layout=Layout(min_height='1000px', min_width='1400px'), map_margin={'top': 50, 'right': 50, 'left': 50, 'bottom': 50}, names=array(['VSS', 'DINT', 'BSCO', ..., 'UGA', 'FNCL', 'EUO'], dtype='<U4'), ref_data=     ticker                                          Fund Name  \
0       SPY                             SPDR S&P 500 ETF Trust   
1       IVV                           iShares Core S&P 500 ETF   
2       VTI                    Vanguard Total Stock Market ETF   
3       VOO                               Vanguard S&P 500 ETF   
4       QQQ                                  Invesco QQQ Trust   
5       VEA                Vanguard FTSE Developed Markets ETF   
6       EFA                              iShares MSCI EAFE ETF   
7      IEFA                         iShares Core MSCI EAFE ETF   
8       VWO                 Vanguard FTSE Emerging Markets ETF   
9       AGG               iShares Core U.S. Aggregate Bond ETF   
10      IJH                       iShares Core S&P Mid-Cap ETF   
11      IWM                           iShares Russell 2000 ETF   
12     IEMG             iShares Core MSCI Emerging Markets ETF   
13      IJR                     iShares Core S&P Small Cap ETF   
14      IWF                    iShares Russell 1000 Growth ETF   
15      VTV                                 Vanguard Value ETF   
16      VUG                                Vanguard Growth ETF   
17      IWD                     iShares Russell 1000 Value ETF   
18      BND                     Vanguard Total Bond Market ETF   
19      LQD  iShares iBoxx USD Investment Grade Corporate B...   
20      XLF                  Financial Select Sector SPDR Fund   
21      VNQ                           Vanguard Real Estate ETF   
22      EEM                  iShares MSCI Emerging Markets ETF   
23      VIG                 Vanguard Dividend Appreciation ETF   
24      GLD                                    SPDR Gold Trust   
25      BSV                       Vanguard Short-Term Bond ETF   
26       VO                               Vanguard Mid-Cap ETF   
27       VB                             Vanguard Small-Cap ETF   
28      TIP                              iShares TIPS Bond ETF   
29      VEU                  Vanguard FTSE All-World ex-US ETF   
...     ...                                                ...   
1616   SFIG  WisdomTree Fundamental U.S. Short-Term Corpora...   
1617   SFHY  WisdomTree Fundamental U.S. Short-Term High Yi...   
1618   HGSD    WisdomTree Global Hedged SmallCap Dividend Fund   
1619    GSD           WisdomTree Global SmallCap Dividend Fund   
1620   WCHN               WisdomTree ICBCCS S&P China 500 Fund   
1621   DWMF          WisdomTree International Multifactor Fund   
1622   DXJF            WisdomTree Japan Hedged Financials Fund   
1623   JHDG  WisdomTree Japan Hedged Quality Dividend Growt...   
1624   GULF               WisdomTree Middle East Dividend Fund   
1625   AGND  WisdomTree Negative Duration U.S. Aggregate Bo...   
1626    QSY     WisdomTree U.S. Quality Shareholder Yield Fund   
1627   SHAG  WisdomTree Yield Enhanced U.S. Short-Term Aggr...   
1628   EQLT                       Workplace Equality Portfolio   
1629   IFIX  Xtrackers Barclays International Corporate Bon...   
1630   IGVT  Xtrackers Barclays International Treasury Bond...   
1631   EMIH  Xtrackers Emerging Markets Bond Interest Rate ...   
1632   EU